In [1]:
import math
import operator
import random
import re
import numpy as np


from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
from scipy.stats import pearsonr
# from gensim.models import KeyedVectors
import pickle

from nltk import cluster


from nltk.cluster import cosine_distance

In [2]:
from contextlib import contextmanager
import sys, os

# To prevent page crashes

@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout

In [3]:
w_names = [
    '1800-w.npy',
    '1810-w.npy',
    '1820-w.npy',
    '1830-w.npy',
    '1840-w.npy',
    '1850-w.npy',
    '1860-w.npy',
    '1870-w.npy',
    '1880-w.npy',
    '1890-w.npy',
    '1900-w.npy',
    '1910-w.npy',
    '1920-w.npy',
    '1930-w.npy',
    '1940-w.npy',
    '1950-w.npy',
    '1960-w.npy',
    '1970-w.npy',
    '1980-w.npy',
    '1990-w.npy'
]

v_names = [
    '1800-vocab.pkl',
    '1810-vocab.pkl',
    '1820-vocab.pkl',
    '1830-vocab.pkl',
    '1840-vocab.pkl',
    '1850-vocab.pkl',
    '1860-vocab.pkl',
    '1870-vocab.pkl',
    '1880-vocab.pkl',
    '1890-vocab.pkl',
    '1900-vocab.pkl',
    '1910-vocab.pkl',
    '1920-vocab.pkl',
    '1930-vocab.pkl',
    '1940-vocab.pkl',
    '1950-vocab.pkl',
    '1960-vocab.pkl',
    '1970-vocab.pkl',
    '1980-vocab.pkl',
    '1990-vocab.pkl'
]

In [4]:
w_keys = [
    '1800',
    '1810',
    '1820',
    '1830',
    '1840',
    '1850',
    '1860',
    '1870',
    '1880',
    '1890',
    '1900',
    '1910',
    '1920',
    '1930',
    '1940',
    '1950',
    '1960',
    '1970',
    '1980',
    '1990'
]

In [29]:
# load word vectors
# Testing with english-fiction diachronic word2vec
# word vectors from https://github.com/williamleif/histwords
wordvecs = {}
vocab = {}
for i, k in enumerate(w_names):
    w = np.load('sgns-eng-fic/' + k)
    v = None
    with open('sgns-eng-fic/' + v_names[i], 'rb') as f:
        v = pickle.load(f)
    wordvecs[w_keys[i]] = w
    vocab[w_keys[i]] = v
    

In [30]:
wordvecs['1810'].size

30000000

In [31]:
vocab['1830'][1000:1005]

['assure', 'jones', 'member', 'observe', 'drink']

In [32]:
# from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.preprocessing import StandardScaler
import sklearn
sklearn.__version__

'0.21.3'

In [153]:
# testv = wordvecs['1810']
# n_clusters=100
# clusters = {}
# for k in w_keys:
#     clusters[k] = AgglomerativeClustering(n_clusters=n_clusters,affinity='euclidean').fit(wordvecs[k][500:10500])

# test_cluster = DBSCAN(eps=0.1, min_samples=3, metric='cosine').fit(testv)

In [155]:
# clustervals = {}
# for k in w_keys:
#     clustervals[k] = clusters[k].labels_


In [33]:
#### Get common vocab
start=500
end=10500

commonvocab = {}
v = set(vocab['1810'][start:end])
for k in w_keys[1:]:
    v = v & set(vocab[k][start:end])
    
commonvocab = {word: [] for word in v}
for word in v:
    for k in w_keys:
        commonvocab[word].append(vocab[k].index(word))

In [34]:
commonlist = list(commonvocab.keys())
with open('commonvocab.pickle', 'wb') as handle:
    pickle.dump(commonvocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [35]:
commonvocab[commonlist[0]]

[8200,
 8200,
 8200,
 8203,
 8210,
 8223,
 8230,
 8237,
 8268,
 8270,
 7593,
 8606,
 7350,
 7122,
 6451,
 6214,
 6429,
 6208,
 5695,
 5639]

In [36]:
commonW = {}
newcommonvocab = {}
for i in range(len(w_keys)):
    commonW[w_keys[i]] = []
for i in range(len(w_keys)):
    for k in commonvocab:
        commonW[w_keys[i]].append(commonvocab[k][i])
        newcommonvocab[k] = len(commonW[w_keys[i]])

In [37]:
newcommonvocab[commonlist[7]]


8

In [38]:
with open('common-vocab-index.pickle', 'wb') as handle:
    pickle.dump(newcommonvocab, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
commonvec = {}
for t in w_keys:
    commonvec[t] = wordvecs[t][commonW[t]]

In [40]:
with open('common-vocab-vectors.pickle', 'wb') as handle:
    pickle.dump(commonvec, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [164]:
# with open('clusters.pickle', 'wb') as handle:
#     pickle.dump(clustervals, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [42]:
commonW['1810'].index(851)

7260

In [45]:
wordvecs['1810']

array([[ 0.0953851 , -0.0102231 ,  0.06517883, ..., -0.04821113,
         0.01816791,  0.02954366],
       [-0.01117909,  0.02249754, -0.08654148, ...,  0.00234814,
         0.05491547,  0.02232076],
       [-0.07287728,  0.00237393, -0.02524905, ...,  0.01331965,
        -0.00164379, -0.00212812],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [47]:
commonvec['1810'][7260]

array([ 9.58853388e-02,  6.62124492e-04,  5.68131834e-02, -3.82639720e-03,
        7.57935817e-02, -5.07373916e-02,  4.60729244e-02,  1.15373532e-01,
        5.59888299e-02, -5.19061455e-02, -4.43256804e-02,  3.85622574e-02,
        8.20031926e-02,  5.81608054e-03, -1.61272544e-02,  1.03402927e-01,
        2.09910884e-03, -4.72073621e-02,  3.78655212e-02,  1.54544999e-03,
       -1.31768124e-01,  2.30348498e-02, -4.85786648e-02,  2.94976385e-02,
       -9.45933728e-02, -3.87269622e-02, -4.29035925e-02,  1.09901244e-01,
       -6.95290764e-02,  2.25496672e-02, -1.86411731e-03,  1.78454350e-02,
       -2.92590623e-02, -2.47242950e-02, -4.83790027e-02,  5.99120082e-02,
       -7.26775029e-02, -5.08610972e-04, -3.46935277e-02, -8.24167387e-02,
       -6.26444674e-02,  5.54035519e-02, -8.05013200e-02, -3.55636644e-02,
        5.01836954e-02,  1.05676756e-01,  1.47137537e-02,  7.27222866e-02,
       -7.12012879e-02,  3.20624628e-03, -2.31758615e-02,  3.10812193e-02,
       -7.96810755e-02, -

In [48]:
n_clusters=100
newclusters = {}
for k in w_keys:
    newclusters[k] = AgglomerativeClustering(n_clusters=n_clusters,affinity='euclidean').fit(commonvec[k])


In [49]:
nlabels=newclusters['1810'].labels_
n_clusters = len(set(nlabels)) - (1 if -1 in nlabels else 0) # Number of clusters in labels
n_clusters

100

In [50]:
with open('clusters.pickle', 'wb') as handle:
    pickle.dump(newclusters, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [54]:
clustsagg = {}
with open('clusters.pickle', 'rb') as f:
    clustsagg = pickle.load(f)

In [53]:
len(clustsagg['1810'].labels_)

7446

In [55]:
clusts = {}
for t in w_keys:
    clusts[t] = clustsagg[t].labels_

In [56]:
commonvec = {}
with open('common-vocab-vectors.pickle', 'rb') as f:
    commonvec = pickle.load(f)

In [57]:
commonvec['1810'][25]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [58]:
centers_dict = {}
for t in w_keys:
    centers_dict[t] = np.zeros((100,300))
    
for t in w_keys:
    cnts = [0] * 100
    for i in range(len(clusts[t])):
        x = clusts[t][i]
        centers_dict[t][x] += commonvec[t][i]
        cnts[x] += 1
    for j in range(100):
        if cnts[j] > 0:
            centers_dict[t] = centers_dict[t]/cnts[j]



In [63]:
with open('cluster-center-vectors.pickle', 'wb') as handle:
    pickle.dump(centers_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
"""
Load from here
"""

In [25]:
# nlabels = clusters['1840'].labels_
# n_clusters = len(set(nlabels)) - (1 if -1 in nlabels else 0) # Number of clusters in labels
# n_clusters

100

In [33]:
# len(vocab['1810'])
# means = wordvecs['1810'][0:20]

In [35]:
# NUM_CLUSTERS = 20
# # clusterer = cluster.EMClusterer(means, bias=0.1)
# clusterer = cluster.KMeansClusterer(NUM_CLUSTERS, cosine_distance, initial_means=means.tolist())
# clusters = clusterer.cluster(wordvecs['1810'][:1000], True)

# print("clustering")
# with suppress_stdout():
#    clusters = clusterer.cluster(wordvecs['1810'][:1000], True, trace=False)
# print("done")

In [40]:
len(clusterer.means())

20

In [42]:
clusters[1]

4